# Training Model YOLO EXPRESS VERSION
**Oleh:** Irfan Muzaki

**Update Terbaru:** 17 Mei 2025



## Memastikan GPU NVIDIA Tersedia

*Rekomendasi GPU: T4*


In [ ]:
!nvidia-smi

# 2.&nbsp;Upload Gambar Dataset dan Pre-processing

## 2.1 Upload Gambar

Upload dataset yang telah dijadikan dalam bentuk `.zip` ke sini dengan mengklik gambar folder di samping kiri ini, bisa seret lepas


**Atau bisa juga Upload dari Google Drive**

Pertama, upload file `.zip` ke Google Drive Anda, dan salin folder tempat Anda menguploadnya. Ganti `MyDrive/..../..../(nama file dataset anda).zip` dengan jalur ke file zip Anda. (Contohnya, saya mengupload file zip ke folder bernama “dataset saya”, jadi saya akan menggunakan `MyDrive/dataset saya/(nama data set saya).zip` untuk jalurnya). Kemudian, jalankan blok kode berikut untuk memasang Google Drive Anda ke sesi Colab ini dan salin folder ke sistem file ini.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/MyDrive/path/to/data.zip /content #<<--------GANTI DENGAN NAMA PATH ANDA

## 2.2 Split Gambar ke Folder "train" dan "validation"

In [ ]:
# Unzip images to a custom data folder
!unzip -q /content/XXXXXXXXXXXXXXXXXX.zip -d /content/custom_data #<GANTI XXXXXXXXXXXXXXX dengan

In [ ]:
!wget -O /content/train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py

# TO DO: Improve robustness of train_val_split.py script so it can handle nested data folders, etc
!python train_val_split.py --datapath="/content/custom_data" --train_pct=0.9

# 3.&nbsp;Install Ultralytics untuk Training


In [ ]:
!pip install ultralytics

# 4.&nbsp;Configure Training


In [ ]:

# Fungsi Python untuk membuat file konfigurasi data.yaml secara otomatis
# 1. Membaca file “classes.txt” untuk mendapatkan daftar nama kelas
# 2. Membuat dictionary data dengan jalur yang benar ke folder, jumlah kelas, dan nama-nama kelas
# 3. Menulis data dalam format YAML ke data.yaml


import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Membaca class.txt untuk mendapatkan nama-nama kelas
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Membuat dictionary data
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Menulis data ke file YAML
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Tentukan jalur ke class.txt dan jalankan fungsi
path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

# 5.&nbsp;Training Model


**Jalankan blok kode berikut untuk memulai pelatihan.**
Jika Anda ingin menggunakan model yang berbeda, jumlah epoch yang berbeda, atau resolusi yang berbeda, ubah nilai dari `model`, `epochs`, atau `imgsz`.



In [ ]:
!yolo detect train data=/content/data.yaml model=yolo11s.pt epochs=60 imgsz=640

**Algoritma pelatihan akan memproses gambar-gambar dalam direktori pelatihan dan validasi, lalu mulai melatih model.**
Di akhir setiap epoch pelatihan, program akan menjalankan model pada dataset validasi dan melaporkan nilai *mAP*, *precision*, dan *recall* yang dihasilkan.
Seiring pelatihan berjalan, *mAP* biasanya akan meningkat di setiap epoch.
Pelatihan akan berakhir setelah melewati jumlah epoch yang ditentukan oleh `epochs`.

> **CATATAN:** Pastikan pelatihan dibiarkan berjalan hingga selesai, karena ada proses *optimizer* yang dijalankan di akhir pelatihan untuk menghapus layer-layer yang tidak diperlukan dari model.

Model terbaik hasil pelatihan akan disimpan di:
`content/runs/detect/train/weights/best.pt`
Informasi tambahan tentang proses pelatihan disimpan dalam folder:
`content/runs/detect/train`
Termasuk file `results.png` yang menunjukkan perkembangan *loss*, *precision*, *recall*, dan *mAP* selama setiap epoch.



#6.&nbsp;Test Model

In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')


## 7. Download Model YOLO



In [ ]:
# Membuat folder "my_model" untuk menyimpan bobot model dan hasil
!mkdir /content/my_model
!cp /content/runs/detect/train/weights/best.pt /content/my_model/my_model.pt
!cp -r /content/runs/detect/train /content/my_model

# Zip menjadi "my_model.zip"
%cd my_model
!zip /content/my_model.zip my_model.pt
!zip -r /content/my_model.zip train
%cd /content

In [ ]:
# Jika tidak mau, download dari samping
from google.colab import files

files.download('/content/my_model.zip')